In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import gmaps
import ast
with open('../.env', 'r') as f:
    key = f.read()
gmaps.configure(api_key= key)

In [2]:
df = pd.read_csv('madrid_houses_raw.csv')

In [3]:
display("Shape : {}".format(df.shape))
display(df.head(20))
display(df.dtypes)
display(df.isnull().sum())
any_null_rows = df.isnull().any(axis=1)
display('Número de filas con nulos: ', len(df[any_null_rows]))

'Shape : (12455, 11)'

,bathrooms,description,floor,location1,location2,price,price_m2,rooms,size,location,position
0,1.0,Coslada junto al recinto ferial. Vivienda de 7...,4ª,Recinto Ferial,Casco Antiguo (Coslada),226.000,2.897,2.0,78.0,"Recinto Ferial, Casco Antiguo (Coslada), Comun...","{'lat': 40.43040939999999, 'lng': -3.5578887}"
1,1.0,"Bajos de 2 dormitorios 50m2, totalmente refor...",0,"calle Isletas, nº 1",Zona Estación (Coslada),98.500,1.790,2.0,55.0,"calle Isletas, nº 1, Zona Estación (Coslada), ...","{'lat': 40.4237329, 'lng': -3.5611868}"
2,1.0,Características del edificio:\n\n- Construido ...,NaN,"calle Mazo, nº 6",Pueblo Nuevo (Distrito Ciudad Lineal. Madrid C...,129.000,1.842,3.0,70.0,"calle Mazo, nº 6, Pueblo Nuevo (Distrito Ciuda...","{'lat': 40.43073589999999, 'lng': -3.6350217}"
3,1.0,REF: 12130 Vivienda exterior localizada en una...,NaN,Vallecas,Palomeras Bajas (Distrito Puente de Vallecas. ...,220.000,3.098,3.0,71.0,"Vallecas, Palomeras Bajas (Distrito Puente de ...","{'lat': 40.384267, 'lng': -3.6630031}"
4,1.0,"Excelente piso reformado en venta, de dos dorm...",0,calle Gomez Acebo,San Andrés (Distrito Villaverde. Madrid Capital),148.000,2.144,2.0,69.0,"calle Gomez Acebo, San Andrés (Distrito Villav...","{'lat': 40.3430374, 'lng': -3.7089705}"
5,1.0,"Tercera planta exterior con ascensor de 87 m2,...",NaN,calle Canarias,Palos de Moguer (Distrito Arganzuela. Madrid C...,300.000,3.448,3.0,87.0,"calle Canarias, Palos de Moguer (Distrito Arga...","{'lat': 40.4018264, 'lng': -3.6923389}"
6,1.0,"Se vende piso en zona de calle Málaga, 85 metr...",NaN,Fuenlabrada - Centro,Avanzada-La Cueva (Fuenlabrada),134.000,1.576,3.0,85.0,"Fuenlabrada - Centro, Avanzada-La Cueva (Fuenl...","{'lat': 40.2930236, 'lng': -3.7857758}"
7,2.0,"INMOLINE, 91 616 32 20 y whatsapp 673 125 727:...",NaN,calle Alcalde Henche de La Plata,Sanchinarro (Distrito Hortaleza. Madrid Capital),450.000,4.090,3.0,110.0,"calle Alcalde Henche de La Plata, Sanchinarro ...","{'lat': 40.48866599999999, 'lng': -3.649591199..."
8,2.0,SITUACION:\n\nLa vivienda se encuentra en la l...,NaN,calle Magnolias,Algete,166.990,2.036,2.0,82.0,"calle Magnolias, Algete, Comunidad de Madrid, ...","{'lat': 40.5985993, 'lng': -3.5000946}"
9,1.0,Vivienda en barrio de Lavapies. Ideal como inv...,0,calle Mesón de Paredes,Embajadores-Lavapiés (Distrito Centro. Madrid ...,140.000,4.000,2.0,35.0,"calle Mesón de Paredes, Embajadores-Lavapiés (...","{'lat': 40.4087204, 'lng': -3.7033913}"


bathrooms      float64
description     object
floor           object
location1       object
location2       object
price           object
price_m2       float64
rooms          float64
size           float64
location        object
position        object
dtype: object

bathrooms       100
description      35
floor          4831
location1         0
location2         0
price             0
price_m2        345
rooms           187
size            225
location          0
position         32
dtype: int64

'Número de filas con nulos: '

5164

Se eliminan en primer lugar las columnas que no nos vayan a servir: Description, location1, location2:

In [4]:
df.drop(['description', 'location1', 'location2', 'location'], axis = 1, inplace = True)

Se observa que la columna de "Price" es de tipo"object" en lugar de numérica, y que la columna price_m2 no está en el orden de
magnitud correcto. Se corrige:

In [5]:
df['price'] = df['price'].apply(lambda x: x.replace('.', ''))
df['price'] = pd.to_numeric(df['price'], errors = 'coerce')
df['price_m2'] = df['price_m2'] * 1000

La columna de 'floor' tampoco es numérica porque las plantas están expresadas como tipo string. Por tanto en primer lugar se eliminan los caracteres de texto y a continuación se convierte en numérica.

In [6]:
df.head(30)
display(df['floor'].unique())

array(['4ª', '0', nan, '2ª', '7ª', '1ª', '9ª', '6ª', '3ª', '5ª', '8ª',
       'planta'], dtype=object)

In [7]:
df['floor'] = df['floor'].apply(lambda x: x.replace('ª', '') if type(x) is str else x)

In [8]:
df.head(10)
display(df['floor'].unique())

array(['4', '0', nan, '2', '7', '1', '9', '6', '3', '5', '8', 'planta'],
      dtype=object)

In [9]:
df['floor'] = pd.to_numeric(df['floor'], errors = 'coerce')

Extraemos la latitud y longitud y las guardarmos en dos columnas. Se elimina la columna 'position'

In [10]:
def convert_str_to_dict(x):
    try:
        return ast.literal_eval(x)
    except:
        return None

df['position'] = df['position'].apply(convert_str_to_dict)
df.loc[:,'latitude'] = df['position'].apply(lambda x: x.get('lat') if x is not None else None)
df.loc[:,'longitude'] = df['position'].apply(lambda x: x.get('lng') if x is not None else None)
df.drop(['position'], axis = 1, inplace = True)

Comprobamos los valores nulos:

In [11]:
nulls = df.isnull().sum()
nulls = nulls[nulls > 0]
display(nulls)

bathrooms     100
floor        4904
price         130
price_m2      345
rooms         187
size          225
latitude       32
longitude      32
dtype: int64

Rellenamos los valores nulos:

En primer lugar seleccionamos las filas que dispongan de:
- Precio y superficie
- superficie y precio por metro cuatrado

In [12]:
df = df[(df['price'].notnull() & df['size'].notnull()) | (df['price_m2'].notnull() & df['size'].notnull())]

Comprobamos nuevamente los valores nulos y vemos que solo se dan para las columnas 'bathrooms', 'floor', 'rooms, 'latitude' y longitude'.

Para 'bathrooms', al ser un número de nulos muy elevado, completamos utilizando el valor medio. Antes de completar los nulos, ñadimos una columna indicador de si el valor estaba presente o no, de forma que el modelo pueda disponer de más información.

Para 'rooms', 'bathrooms', 'latitude' y 'longitude', eliminamos las filas al ser un valor reducido.

In [13]:
df = df[df['bathrooms'].notnull() & df['rooms'].notnull() & df['latitude'].notnull() & df['longitude'].notnull()]

In [14]:
df['floor_was_missing'] = df['floor'].isnull()
df['floor'] = df['floor'].fillna(df['floor'].mean())

Para verificar la posición de los datos, representamos su ubicación en un mapa.

In [15]:
fig = gmaps.figure(map_type='SATELLITE')
data_loc = df[['latitude', 'longitude']]
heatmap_layer = gmaps.heatmap_layer(data_loc)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Se observa que la mayoría de los valores están ubicados en Madrid de forma correcta, pero en un ánalisis de Tableau se observan muchos puntos en otros países. Para realizar una medida exacta, filtraremos por valores de coordenadas, asignando valores nulos a los puntos que estén fuera de la comunidad de Madrid:
- Rango de latitudes : [40.03, 41.08]
- Rango de longitudes : [-4.5, -3.5]

In [16]:
madrid_data = df[(df['latitude'] > 40.03) & (df['latitude'] < 41.08) & \
                 (df['longitude'] > -4.5) & (df['longitude'] < -3.5)]

In [17]:
df = df.reset_index()
rows_to_convert = df[(df['latitude'] < 39.9) | (df['latitude'] > 41.16) | \
                 (df['longitude'] < -4.57) | (df['longitude'] > -3.06)].index

In [18]:
df.loc[df.index.isin(rows_to_convert), ['latitude', 'longitude']] = None

In [19]:
df['price_cut'] = pd.qcut(df.price,20)

In [20]:
df['latitude'] = df.groupby(['price_cut'])['latitude'].transform(lambda x : x.fillna(x.mean()))

In [21]:
df['longitude'] = df.groupby(['price_cut'])['longitude'].transform(lambda x : x.fillna(x.mean()))

In [22]:
df = df.drop_duplicates()
df.drop('index', axis = 1, inplace = True)

Visualizamos los resultados en un mapa:

In [ ]:
data_loc = df[['latitude', 'longitude']]

houses_layer = gmaps.symbol_layer(
    data_loc, fill_color="green", stroke_color="red", scale=2
)

fig = gmaps.figure()
fig.add_layer(houses_layer)
display(fig)

Para finalizar, pintamos la matriz de correlación para comprobar que no exiten alta colinealidad entre dos columnas

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(12,7))
sns.heatmap(corr_matrix, annot = True, linewidth = 0.5)

In [ ]:
df = df.drop_duplicates()
df.to_csv('data_wrangled.csv', index = None)